<a href="https://colab.research.google.com/github/kartikeyahl/MS_analysis/blob/main/PunchTimeAnalysis11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from datetime import datetime
import math

In [2]:
# Importing the excel
!unzip punch_short.xlsx
dataset = pd.read_excel('from 9th.xlsx',parse_dates=['Date'],
    date_parser=lambda x: pd.to_datetime(x, format='%Y%m%d'))
dataset2=pd.read_excel('Master Data.xlsx')

unzip:  cannot find or open punch_short.xlsx, punch_short.xlsx.zip or punch_short.xlsx.ZIP.


In [3]:
#Renaming column name to a valid column name format as per python
dataset.rename(columns={'IN/OUT': 'IN_OUT'}, inplace=True)

In [4]:
#Droping rows that conatain 1 in 'IN_OUT' column
dataset=dataset[dataset.IN_OUT!=1]

In [5]:
#Converting Date column values into year-month-day format
# j,k=0,0
# for i in dataset.iloc[:,3]:
#   dataset.Date[j]=datetime.strptime(str(i),'%Y%m%d').date()
#   j+=1

dataset['Date'] = pd.to_datetime(dataset['Date'], errors='coerce')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [6]:
a1=str(dataset.iloc[0]['Date'])
a2=str(dataset.iloc[-1]['Date'])
d1 = datetime.strptime(a1[0:10], "%Y-%m-%d")
d2 = datetime.strptime(a2[0:10], "%Y-%m-%d")
date_diff = abs((d2 - d1).days)+1  

In [ ]:
#Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)
l=[]
for i in range(len(dataset)-1):
  j=i+1
  x=str(dataset.iloc[i]['Time'])
  y=str(dataset.iloc[j]['Time'])

  if len(x)<=5:
    x=x.zfill(6)
  if len(y)<=5:
    y=y.zfill(6)

  x = datetime.strptime(x, "%H%M%S")
  x.strftime("%I:%M %p")
  y = datetime.strptime(y, "%H%M%S")
  y.strftime("%I:%M %p")

  l.append([dataset.iloc[i]['E.Code'], dataset.iloc[i]['Date'].date(), dataset.iloc[i]['IN_OUT'], int(dataset.iloc[i]['Time']), datetime.combine(dataset.iloc[j]['Date'], y.time()) - datetime.combine(dataset.iloc[i]['Date'], x.time())])  
  i=i+2

In [ ]:
#Replacing ":" with "." and getting upto 2 decimal places in Time to make mathematical operations feasible on i
for i in range(len(l)):
  l[i][4] = l[i][4].__str__().replace(":",".")
  s=l[i][4]
  l[i][4]=s[0:-3]

In [ ]:
#Removing redundant rows and storing in new data-structure
l2=[]
for i in range(len(l)):
  if 'P20' not in str(l[i][2]):
    l2.append(l[i]) 

In [ ]:
for i in range(len(l2)):
  if 'd' in l2[i][4]:
    s=l2[i][4]
    s=float(s[8:len(s)])
    l2[i][4]=24+s

In [ ]:
# s='1 day, 0.53'

# s=float(s[5:len(s)])
# s=24+s
# print(s)

In [ ]:
#Converting string type to float for Time column
for i in range(len(l2)):
  try:
   l2[i][4]=float("{:.2f}".format(float(l2[i][4])))
  except:
    print(l2[i][0],l2[i][4])

In [ ]:
l2

In [ ]:
"""Converting time from float to mins, applied logic to check if it is greater than 9hrs & 30mins. Subtacting the 
result to get over time (minutes) converting and storing it into hrs.minutes format. """

l3=[]
for i in range(len(l2)):
  
  frac, whole = math.modf(l2[i][4])
  mins=int(whole*60+frac*100)
  
  if l2[i][2]=='P10' and l2[i][3]>220000:
    # print(frac,whole)
    # print(l2[i][0])
    # print(l2[i][4])
    # print(mins)
    if mins>450:
      mins=mins-450
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

  else:
    mins=mins-570
    hours=mins//60
    minutes=mins%60
    if minutes<10:
      ot_time = float("{}.0{}".format(hours, minutes))
    elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
    else:
      ot_time = float("{}.{}".format(hours, minutes))
    l2[i].append(ot_time)
    l3.append(l2[i])  

In [ ]:
89%10

In [ ]:
#Removing redundant rows and storing in new data-structure
l4=[]
for i in range(len(l3)):
  if l3[i][5]>0:
    l4.append(l3[i]) 

In [ ]:
# Converting 2d list to dataframe to apply CRUD opertations
df = pd.DataFrame(l4, columns=['E.Code','Date','IN/OUT','Time','Total Time','OT'])

In [ ]:
#Droping an unnecesary column
df=df.drop(['IN/OUT','Time','Total Time'], axis=1)

In [ ]:
df

In [ ]:
ll1=[]
def binarySearch(arr, l, r, x):

	while l <= r:

		mid = l + (r - l) // 2

		# Check if x is present at mid
		if arr[mid] == x:
			return mid

		# If x is greater, ignore left half
		elif arr[mid] < x:
			l = mid + 1

		# If x is smaller, ignore right half
		else:
			r = mid - 1

	# If we reach here, then the element
	# was not present
	return -1


# Driver Code
arr = [int(i) for i in dataset2['E Code']]

for i in range(len(df)):
  # Function call
  result = binarySearch(arr, 0, len(arr)-1, df.iloc[i]['E.Code'])

  if result != -1:
      ll1.append([dataset2.iloc[result]['Full Name'], dataset2.iloc[result]['Designation'],dataset2.iloc[result]['Operation'],dataset2.iloc[result]['Division'],dataset2.iloc[result]['Department']])
  else:
      ll1.append(['NaN','NaN','NaN','NaN'])


In [ ]:
# Converting 2d list to dataframe to apply CRUD opertations
dff = pd.DataFrame(ll1, columns=['Full Name','Designation','Operation','Division','Department'])

In [ ]:
df_final = pd.concat([df['E.Code'], dff, df[['Date','OT']]], axis=1)

In [ ]:
""" Printing dataframe (E.Code, Designation, Opr., Division, Dept., Date, OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes)) 
"""
df_final

In [ ]:
# lx=[]
# for i in df_final['Operation']:
#   if i not in lx:
#     lx.append(i)


# for i in lx:
#   c,d=1,0
#   lxc=[]
#   for j in df_final['Operation']:
#     if i == j:
#       c+=1
#   lxc.append([i,c])
#   lx.append(lxc)

In [ ]:
# df_final.iloc[10035]

In [ ]:
""" Applying Groupby opr. to group dataframe on basis of E.Code and apply sum and count opr. to 
compute Total_OT & Total_OT_Days """

groupby_ecode_Total_OT = df.groupby(['E.Code'],as_index=False).sum()
groupby_ecode_Total_OT_Days = df.groupby(['E.Code'],as_index=False).count()
groupby_operation_sum = df_final.groupby(['Operation'],as_index=False).sum()
groupby_operation = df_final.groupby(['Operation'],as_index=False).count()
groupby_division_sum = df_final.groupby(['Division'],as_index=False).sum()
groupby_division = df_final.groupby(['Division'],as_index=False).count()
groupby_designation_sum = df_final.groupby(['Designation'],as_index=False).sum()
groupby_designation = df_final.groupby(['Designation'],as_index=False).count()


In [ ]:
Operation_count = pd.DataFrame(df_final.groupby(['Operation'],as_index=False).size())
Division_count = pd.DataFrame(df_final.groupby(['Division'],as_index=False).size())
Designation_count = pd.DataFrame(df_final.groupby(['Designation'],as_index=False).size())

In [ ]:
#Converting Groupby object into DataFrame to apply CRUD opr.
df2=pd.DataFrame(groupby_ecode_Total_OT)
df3=pd.DataFrame(groupby_ecode_Total_OT_Days)
df4=pd.DataFrame(groupby_operation)
df5=pd.DataFrame(groupby_division)
df6=pd.DataFrame(groupby_operation_sum)
df7=pd.DataFrame(groupby_division_sum)
df8=pd.DataFrame(groupby_designation)
df9=pd.DataFrame(groupby_designation_sum)

In [ ]:
df4=df4.drop(['E.Code','Full Name','Designation','Division','Department','Date'], axis=1)
df5=df5.drop(['E.Code','Full Name','Designation','Operation','Department','Date'], axis=1)
df8=df8.drop(['E.Code','Full Name','Operation','Division','Department','Date'], axis=1)
df6=df6.drop(['E.Code'], axis=1)
df7=df7.drop(['E.Code'], axis=1)
df9=df9.drop(['E.Code'], axis=1)

In [ ]:
df4.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df5.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
df8.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df6.rename(columns={'OT': 'Total OT_Days'}, inplace=True)
# df7.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [ ]:
l4=[]
for i in df2.index:
      frac, whole = math.modf(df2['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l4.append(ot_time)

In [ ]:
llp=[]
for i in df6.index:
      frac, whole = math.modf(df6['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      llp.append(ot_time)

In [ ]:
lllp=[]
for i in df7.index:
      frac, whole = math.modf(df7['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      lllp.append(ot_time)

In [ ]:
llllp=[]
for i in df9.index:
      frac, whole = math.modf(df9['OT'][i])
      mins=int(whole*60+frac*100)
      hours=mins//60
      minutes=mins%60
      if minutes<10:
          ot_time = float("{}.0{}".format(hours, minutes))
      elif minutes%10==0:
        ot_time = float("{}.{}0".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      llllp.append(ot_time)

In [ ]:
df2['Total OT']=l4
df6['Total OT']=llp
df7['Total OT']=lllp
df9['Total OT']=llllp

In [ ]:
#Droping an unnecesary column
df3=df3.drop(['Date'], axis=1)

In [ ]:
df2=df2.drop(['OT'], axis=1)
df6=df6.drop(['OT'], axis=1)
df7=df7.drop(['OT'], axis=1)
df9=df9.drop(['OT'], axis=1)

In [ ]:
#Renaming Columns
#df2.rename(columns={'OT': 'Total OT'}, inplace=True)
df3.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [ ]:
lp=[]
for i in range(len(df6)):
  lp.append(df6.iloc[i]['Total OT']/df4.iloc[i]['Total OT_Days'])
lp2=[]
for i in range(len(df7)):
  lp2.append(df7.iloc[i]['Total OT']/df5.iloc[i]['Total OT_Days'])
lp3=[]
for i in range(len(df9)):
  lp3.append(df9.iloc[i]['Total OT']/df8.iloc[i]['Total OT_Days'])

In [ ]:
lp=pd.DataFrame(lp,columns=['Average OT'])
lp2=pd.DataFrame(lp2,columns=['Average OT'])
lp3=pd.DataFrame(lp3,columns=['Average OT'])

In [ ]:
lp

In [ ]:
#Creating a final dataframe (E.Code, Total OT_Days, Total OT)
# result = pd.concat([df3,df2], axis=1)
result_2 =  pd.concat([df4[['Operation','Total OT_Days']],df6['Total OT'],Operation_count['size'],lp['Average OT']], axis=1)
result_3 =  pd.concat([df5,df7['Total OT'],Division_count['size'],lp2['Average OT']], axis=1)
result_4 =  pd.concat([df8,df9['Total OT'],Designation_count['size'],lp3['Average OT']], axis=1)

In [ ]:
result_2.rename(columns={'size': 'Count'}, inplace=True)
result_3.rename(columns={'size': 'Count'}, inplace=True)
result_4.rename(columns={'size': 'Count'}, inplace=True)

In [ ]:
ll2=[]
def binarySearch(arr, l, r, x):

	while l <= r:

		mid = l + (r - l) // 2

		# Check if x is present at mid
		if arr[mid] == x:
			return mid

		# If x is greater, ignore left half
		elif arr[mid] < x:
			l = mid + 1

		# If x is smaller, ignore right half
		else:
			r = mid - 1

	# If we reach here, then the element
	# was not present
	return -1


# Driver Code
arr = [int(i) for i in dataset2['E Code']]

for i in range(len(df3)):
  # Function call
  res = binarySearch(arr, 0, len(arr)-1, df3.iloc[i]['E.Code'])

  if res != -1:
      ll2.append([dataset2.iloc[res]['Full Name'], dataset2.iloc[res]['Designation'],dataset2.iloc[res]['Operation'],dataset2.iloc[res]['Division'],dataset2.iloc[res]['Department']])
  else:
      ll2.append(['NaN','NaN','NaN','NaN'])


In [ ]:
# Converting 2d list to dataframe to apply CRUD opertations
dfff = pd.DataFrame(ll2, columns=['Full Name','Designation','Operation','Division','Department'])

In [ ]:
result = pd.concat([df3['E.Code'], dfff, df3['Total OT_Days'], df2['Total OT']], axis=1)

In [ ]:
""" Printing dataframe (E.Code, Total OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes), 
    Total OT_Days) 
"""
result

In [ ]:
l_diff=[]
if date_diff==88 or 89 or 90 or 91 or 92:
  for i in range(len(result)):
    if result.iloc[i]['Total OT']>50:
      l_diff.append([result.iloc[i]['E.Code'],result.iloc[i]['Full Name'],result.iloc[i]['Designation'],result.iloc[i]['Operation'],result.iloc[i]['Division'],result.iloc[i]['Department'],result.iloc[i]['Total OT_Days'],result.iloc[i]['Total OT']])
elif date_diff==28 or 29 or 30 or 31:
  for i in range(len(result)):
    if result.iloc[i]['Total OT']>16:
      l_diff.append([result.iloc[i]['E.Code'],result.iloc[i]['Full Name'],result.iloc[i]['Designation'],result.iloc[i]['Operation'],result.iloc[i]['Division'],result.iloc[i]['Department'],result.iloc[i]['Total OT_Days'],result.iloc[i]['Total OT']])
else:
  for i in range(len(result)):
    if result.iloc[i]['Total OT']>((16/30)*date_diff):
      l_diff.append([result.iloc[i]['E.Code'],result.iloc[i]['Full Name'],result.iloc[i]['Designation'],result.iloc[i]['Operation'],result.iloc[i]['Division'],result.iloc[i]['Department'],result.iloc[i]['Total OT_Days'],result.iloc[i]['Total OT']])


In [ ]:
l_diff = pd.DataFrame(l_diff, columns=['E.Code','Full Name','Designation','Operation','Division','Department','Total OT_Days','Total OT'])

In [ ]:
l_diff

In [ ]:
# consecutive_days=[]
# employees_covered=[]
# for i in range(len(df_final)):
#   count=1
#   if df_final.iloc[i]['E.Code'] not in employees_covered:
#     employees_covered.append([i,df_final.iloc[i]['E.Code']])
#     for j in range(i+1,len(df_final)):
#       if df_final.iloc[i]['Date'] ==df_final.iloc[j]['Date']:
#         count+=1
#     if count>1:
#       consecutive_days.append([df_final.iloc[i]['E.Code'],df_final.iloc[i]['Full Name'],df_final.iloc[i]['Designation'],df_final.iloc[i]['Operation'],df_final.iloc[i]['Division'],df_final.iloc[i]['Department']])


In [ ]:
# consecutive_days = pd.DataFrame(consecutive_days, columns=['E.Code','Full Name','Designation','Operation','Division','Department'])

In [ ]:
# Sorting dataframe in descending order on basis of Total OT_Days (User can get top x employees on basis of Total OT_Days)
cont2=result.sort_values(by=['Total OT_Days'], ascending=False)

In [ ]:
cont2_consecutive=[]
i=0
while cont2.iloc[i]['Total OT_Days']>=10:
  cont2_consecutive.append(cont2.iloc[i]['E.Code'])
  i+=1

In [ ]:
cont2_consecutive
l_con=[]
for x in cont2_consecutive:
  for j in range(len(df_final)):
    if x == df_final.iloc[j]['E.Code']:
      l_con.append([x, df_final.iloc[j]['Date']])


In [ ]:
l_10con=[]
for j in cont2_consecutive:
  p,c=0,1
  for i in range(len(l_con)):
    p=i+1
    if p <= len(l_con)-1:
      if j==l_con[i][0] and j==l_con[p][0]:
          if l_con[i][1].day-l_con[p][1].day==-1:
            c+=1
          elif l_con[i][1].weekday()==4:
            if l_con[i][1].day-l_con[p][1].day==-3:
              c+=1
  if c>=10:
    l_10con.append(j)

In [ ]:
l_10_final=[]
# result_10day=result
# result_10day=result_10day.drop(['Total OT_Days','Total OT'],axis=1)
j=0
while j<len(l_10con):
  for i in range(len(result)):
    if result.iloc[i]['E.Code']== l_10con[j]:
      l_10_final.append([result.iloc[i]['E.Code'],result.iloc[i]['Full Name'],result.iloc[i]['Designation'],result.iloc[i]['Operation'],result.iloc[i]['Division'],result.iloc[i]['Department']])
      break;
  j+=1

dfff_10 = pd.DataFrame(l_10_final, columns=['E.Code','Full Name','Designation','Operation','Division','Department'])


In [ ]:
dfff_10

In [ ]:
# Sorting dataframe in descending order on basis of Total OT (User can get top x employees on basis of Total OT)
cont3=result.sort_values(by=['Total OT'], ascending=False)

In [ ]:
cont4=result_2.sort_values(by=['Total OT_Days'], ascending=False)

In [ ]:
cont5=result_2.sort_values(by=['Total OT'], ascending=False)

In [ ]:
cont6=result_3.sort_values(by=['Total OT_Days'], ascending=False)

In [ ]:
cont7=result_3.sort_values(by=['Total OT'], ascending=False)

In [ ]:
cont8=result_4.sort_values(by=['Total OT_Days'], ascending=False)

In [ ]:
cont9=result_4.sort_values(by=['Total OT'], ascending=False)

In [ ]:
!pip install xlsxwriter

In [ ]:
from openpyxl import Workbook
import xlsxwriter
import os

workbook = xlsxwriter.Workbook(r'OT_Analysis.xlsx')
with pd.ExcelWriter(r'OT_Analysis.xlsx') as writer:
#writer = r"C:\Users\Kartikey\Desktop\shorttt.xlsx"
# write dataframe to excel
    df_final.to_excel(writer,sheet_name='OT')
    result.to_excel(writer,sheet_name='Total_OT')
    cont2.to_excel(writer,sheet_name='Highest_Total_OT_Days')
    cont3.to_excel(writer,sheet_name='Highest_Total_OT')
    l_diff.to_excel(writer,sheet_name='Exceeding OT Limit')
    dfff_10.to_excel(writer,sheet_name='Exceeding 10 cons. OT days')
    cont4.to_excel(writer,sheet_name='Operation_Highest_Total_OT_Days')
    cont5.to_excel(writer,sheet_name='Operation_Highest_Total_OT')
    cont6.to_excel(writer,sheet_name='Division_Highest_Total_OT_Days')
    cont7.to_excel(writer,sheet_name='Division_Highest_Total_OT')
    cont8.to_excel(writer,sheet_name='Desig_Highest_Total_OT_Days')
    cont9.to_excel(writer,sheet_name='Desig_Highest_Total_OT')